In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
import time
from sklearn.externals import joblib
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data_path = '../../data/output/normalized_data_X_5p.csv'
y_path = '../../data/output/y_5p.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(69884, 420)
(69884, 1)


,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.0,0.084945,0.264128,0.439400,0.010487,0.0,0.000006,0.265656,0.439493,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.118068,0.0,0.084965,0.264428,0.443488,0.014304,0.0,0.000009,0.265956,0.444051,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.0,0.084972,0.263625,0.448606,0.018030,0.0,0.000010,0.264915,0.448957,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.118068,0.0,0.084979,0.261608,0.458297,0.025182,0.0,0.000014,0.263352,0.456961,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.118068,0.0,0.084985,0.260228,0.463266,0.028953,0.0,0.000000,0.261792,0.463170,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [3]:
y['busBunching1'] = 0
y['busBunching2'] = 0
y['busBunching3'] = 0
y['busBunching4'] = 0
y['busBunching5'] = 0

In [4]:
for i in range(data.shape[1]-5):
    
    current_bus_code = data['busCode'][i]
    current_bus_codeSB = data['busCodeSB'][i]
    
    if ((data['busCode'][i+1] == current_bus_code) & (data['busCodeSB'][i+1] == current_bus_codeSB)):
        y['busBunching1'][i] = y['busBunching'][i+1]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+2] == current_bus_code) & (data['busCodeSB'][i+2] == current_bus_codeSB)):
        y['busBunching2'][i] = y['busBunching'][i+2]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+3] == current_bus_code) & (data['busCodeSB'][i+3] == current_bus_codeSB)):
        y['busBunching3'][i] = y['busBunching'][i+3]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+4] == current_bus_code) & (data['busCodeSB'][i+4] == current_bus_codeSB)):
        y['busBunching4'][i] = y['busBunching'][i+4]
    #else:
        #what to put when does not exists?
    
    if ((data['busCode'][i+5] == current_bus_code) & (data['busCodeSB'][i+5] == current_bus_codeSB)):
        y['busBunching5'][i] = y['busBunching'][i+5]
    #else:
        #what to put when does not exists?
    

/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/veruska/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/veruska/.local/lib/python3.6/site-packages/

In [5]:
y['busBunching1'][y['busBunching1'] == 0]

0          0
1          0
2          0
3          0
4          0
          ..
1432628    0
1432629    0
1432630    0
1432631    0
1432632    0
Name: busBunching1, Length: 1432630, dtype: int64

In [6]:
data[['busCode','busCodeSB','stopPointId']].head(10)

,busCode,busCodeSB,stopPointId
0,0.0,0.000014,0.337464
1,0.0,0.000014,0.337602
2,0.0,0.000014,0.337739
3,0.0,0.000014,0.337877
4,0.0,0.000014,0.297398
5,0.0,0.000014,0.260774
6,0.0,0.000014,0.260498
7,0.0,0.000014,0.260223
8,0.0,0.000014,0.259948
9,0.0,0.000014,0.259672


In [5]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

### Ensemble Multi-Step (5-step)

In [ ]:
# preparar a base, adicionar outros 5ys (5 predições a frente)
# retreinar os modelos
# executar ensemble
# calcular acurácia

In [6]:
knn_model = KNeighborsClassifier(n_jobs=-1)

rf_model = RandomForestClassifier(n_estimators=100, min_samples_split=5,
                           max_features=0.8, n_jobs=-1)

xgb_model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=120, 
 max_depth=50,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=1,
 seed=23)

In [7]:
ensemble_rf_xgb_knn = VotingClassifier([('rf', rf_model), ('xgb', xgb_model), ('knn', knn_model)])

# 1 day
# Accuracy: 0.9653001359376118, 0.9996422694426558, 0.9997853616655935, 0.9995707233311869, 0.9997138155541246, 0.9997853616655935
# Precision: 0.9405241935483871, 0.42857142857142855, 1.0, 1.0, 0.75, 1.0
# Recall: 0.6865342163355408, 0.75, 0.5, 0.25, 0.5, 0.4
# F-measure: 0.7937048064653338, 0.5454545454545454, 0.6666666666666666, 0.4, 0.6, 0.5714285714285715

In [7]:
ensemble_rf_knn_xgb = VotingClassifier([('rf', rf_model), ('knn', knn_model), ('xgb', xgb_model)])

# 1 day
# Accuracy: 0.9655863203834871, 0.9996422694426558, 0.9997138155541246, 0.9997138155541246, 0.9997138155541246, 0.9997853616655935
# Precision: 0.9407630522088354, 0.42857142857142855, 0.75, 1.0, 0.75, 1.0
# Recall: 0.6894775570272259, 0.75, 0.5, 0.5, 0.5, 0.4
# F-measure: 0.7957537154989384, 0.5454545454545454, 0.6, 0.6666666666666666, 0.6, 0.5714285714285715

In [7]:
# MELHOR
ensemble_xgb_knn_rf = VotingClassifier([('xgb', xgb_model), ('knn', knn_model), ('rf', rf_model)])

# 1 day
# Accuracy: 0.9660155970523002, 0.9996422694426558, 0.9996422694426558, 0.9996422694426558, 0.9997853616655935, 0.9997853616655935
# Precision: 0.9428857715430862, 0.42857142857142855, 0.6666666666666666, 1.0, 1.0, 1.0
# Recall: 0.6924208977189109, 0.75, 0.3333333333333333, 0.375, 0.5, 0.4
# F-measure: 0.7984726347051336, 0.5454545454545454, 0.4444444444444444, 0.5454545454545454, 0.6666666666666666, 0.5714285714285715

In [7]:
ensemble_xgb_rf_knn = VotingClassifier([('xgb', xgb_model), ('rf', rf_model), ('knn', knn_model)])

# 1 day
# Accuracy: 0.9655863203834871, 0.9996422694426558, 0.9997853616655935, 0.9995707233311869, 0.9997853616655935, 0.9997853616655935
# Precision: 0.9416498993963782, 0.42857142857142855, 1.0, 1.0, 1.0, 1.0
# Recall: 0.6887417218543046, 0.75, 0.5, 0.25, 0.5, 0.4
# F-measure: 0.7955801104972375, 0.5454545454545454, 0.6666666666666666, 0.4, 0.6666666666666666, 0.5714285714285715

In [7]:
ensemble_knn_rf_xgb = VotingClassifier([('knn', knn_model), ('rf', rf_model), ('xgb', xgb_model)])

# 1 day
# Accuracy: 0.9653001359376118, 0.9996422694426558, 0.9997138155541246, 0.9996422694426558, 0.9997853616655935, 0.9997853616655935
# Precision: 0.9387550200803213, 0.42857142857142855, 0.75, 1.0, 1.0, 1.0
# Recall: 0.6880058866813834, 0.75, 0.5, 0.375, 0.5, 0.4
# F-measure: 0.7940552016985138, 0.5454545454545454, 0.6, 0.5454545454545454, 0.6666666666666666, 0.5714285714285715

In [7]:
ensemble_knn_xgb_rf = VotingClassifier([('knn', knn_model), ('xgb', xgb_model), ('rf', rf_model)])

# 1 day
# Accuracy: 0.9654432281605495, 0.9996422694426558, 0.9997138155541246, 0.9995707233311869, 0.9997138155541246, 0.9997853616655935
# Precision: 0.9424242424242424, 0.42857142857142855, 0.75, 1.0, 0.75, 1.0
# Recall: 0.6865342163355408, 0.75, 0.5, 0.25, 0.5, 0.4
# F-measure: 0.7943805874840357, 0.5454545454545454, 0.6, 0.4, 0.6, 0.5714285714285715

In [8]:
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_model.fit(train_X, train_Y)
knn_pred = knn_model.predict(test_X)
knn_pred

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
knn_model = KNeighborsClassifier(n_jobs=-1)
knn_model.fit(train_X, train_Y)
knn_pred = knn_model.predict(test_X)
knn_pred

In [ ]:
rf = joblib.load('../RandomForest/Saved_RF_100_5_08_BB_class.pkl')
xgb = joblib.load('../XGBoost/Saved_XGB_BB_class.pkl')
knn = joblib.load('../KNN/Saved_KNN_5_distance.pkl')

rf_pred = rf.predict(test_X) #train_X
xgb_pred = xgb.predict(test_X)
knn_pred = knn.predict(test_X)


final_predictions = []

for i in range(0, len(rf_pred)):
    temp_preds = []
    temp_preds.append(rf_pred[i])
#     temp_preds.append(gb_pred[i])
    temp_preds.append(xgb_pred[i])
    temp_preds.append(knn_pred[i])
    
    final_predictions.append(
        max(temp_preds,key=temp_preds.count)
    )

In [ ]:
start = time.time()

multi_target_forest = MultiOutputClassifier(ensemble_xgb_knn_rf)
multi_target_forest.fit(train_X, train_Y)

end = time.time()
print("Prediction execution time: " + str((end - start)/60) + " min")

In [ ]:
start_p = time.time()

pred = multi_target_forest.predict(test_X)

end_p = time.time()
print("Prediction execution time: " + str((end_p - start_p)/60) + " min")

In [ ]:
test_Y.head(10)

In [ ]:
pred

In [ ]:
# Multi-step: 0
accuracy_0 = accuracy_score(test_Y['busBunching'], pred[:,0])
precision_0 = precision_score(test_Y['busBunching'], pred[:,0])
recall_0 = recall_score(test_Y['busBunching'], pred[:,0])
f_measure_0 = f1_score(test_Y['busBunching'], pred[:,0])

# Multi-step: 1
accuracy_1 = accuracy_score(test_Y['busBunching1'], pred[:,1])
precision_1 = precision_score(test_Y['busBunching1'], pred[:,1])
recall_1 = recall_score(test_Y['busBunching1'], pred[:,1])
f_measure_1 = f1_score(test_Y['busBunching1'], pred[:,1])

# Multi-step: 2
accuracy_2 = accuracy_score(test_Y['busBunching2'], pred[:,2])
precision_2 = precision_score(test_Y['busBunching2'], pred[:,2])
recall_2 = recall_score(test_Y['busBunching2'], pred[:,2])
f_measure_2 = f1_score(test_Y['busBunching2'], pred[:,2])

# Multi-step: 3
accuracy_3 = accuracy_score(test_Y['busBunching3'], pred[:,3])
precision_3 = precision_score(test_Y['busBunching3'], pred[:,3])
recall_3 = recall_score(test_Y['busBunching3'], pred[:,3])
f_measure_3 = f1_score(test_Y['busBunching3'], pred[:,3])

# Multi-step: 4
accuracy_4 = accuracy_score(test_Y['busBunching4'], pred[:,4])
precision_4 = precision_score(test_Y['busBunching4'], pred[:,4])
recall_4 = recall_score(test_Y['busBunching4'], pred[:,4])
f_measure_4 = f1_score(test_Y['busBunching4'], pred[:,4])

# Multi-step: 5
accuracy_5 = accuracy_score(test_Y['busBunching5'], pred[:,5])
precision_5 = precision_score(test_Y['busBunching5'], pred[:,5])
recall_5 = recall_score(test_Y['busBunching5'], pred[:,5])
f_measure_5 = f1_score(test_Y['busBunching5'], pred[:,5])

In [ ]:
print("Accuracy: " + str(accuracy_0) + ', ' + str(accuracy_1) + ', ' + str(accuracy_2) + ', ' + str(accuracy_3)
      + ', ' + str(accuracy_4) + ', ' + str(accuracy_5))
print("Precision: " + str(precision_0) + ', ' + str(precision_1) + ', ' + str(precision_2) + ', ' + str(precision_3)
      + ', ' + str(precision_4) + ', ' + str(precision_5))
print("Recall: " + str(recall_0) + ', ' + str(recall_1) + ', ' + str(recall_2) + ', ' + str(recall_3)
      + ', ' + str(recall_4) + ', ' + str(recall_5))
print("F-measure: " + str(f_measure_0) + ', ' + str(f_measure_1) + ', ' + str(f_measure_2) + ', ' + str(f_measure_3)
      + ', ' + str(f_measure_4) + ', ' + str(f_measure_5))